In [75]:
import pandas as pd
import sqlite3
from datetime import datetime, date

conn = sqlite3.connect('lahman.sqlite')

query = "SELECT * FROM sqlite_master;"

df_schema = pd.read_sql_query(query, conn)

df_schema.tbl_name.unique()

array(['allstarfull', 'appearances', 'awardsmanagers', 'awardsplayers',
       'awardssharemanagers', 'awardsshareplayers', 'batting',
       'battingpost', 'collegeplaying', 'divisions', 'fielding',
       'fieldingof', 'fieldingofsplit', 'fieldingpost', 'halloffame',
       'homegames', 'leagues', 'managers', 'managershalf', 'parks',
       'people', 'pitching', 'pitchingpost', 'salaries', 'schools',
       'seriespost', 'teams', 'teamsfranchises', 'teamshalf'],
      dtype=object)

#### Understanding what's in different tables

In [87]:
query = "SELECT * FROM salaries"

check_table = pd.read_sql_query(query, conn)
check_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26428 entries, 0 to 26427
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   ID        26428 non-null  int64  
 1   yearID    26428 non-null  int64  
 2   teamID    26428 non-null  object 
 3   team_ID   26428 non-null  int64  
 4   lgID      26428 non-null  object 
 5   playerID  26428 non-null  object 
 6   salary    26428 non-null  float64
dtypes: float64(1), int64(3), object(3)
memory usage: 1.4+ MB


<br>

#### 1. What was the total spent on salaries by each team, each year?

In [9]:
query = """
SELECT salaries.yearID, teams.name AS Team_Name, sum(salaries.salary) as Total_Salaries 
FROM salaries LEFT JOIN teams
ON salaries.teamID = teams.teamID
GROUP BY salaries.yearID, teams.name
"""

salary_team_year = pd.read_sql_query(query, conn)
salary_team_year

,yearID,Team_Name,Total_Salaries
0,1985,Atlanta Braves,7.995780e+08
1,1985,Baltimore Orioles,7.630070e+08
2,1985,Boston Americans,7.628292e+07
3,1985,Boston Red Sox,1.220527e+09
4,1985,California Angels,4.616926e+08
...,...,...,...
1436,2016,Tampa Bay Rays,6.851677e+08
1437,2016,Texas Rangers,8.449859e+09
1438,2016,Toronto Blue Jays,5.964173e+09
1439,2016,Washington Nationals,2.124790e+09


<br>

#### 2. What is the first and last year played for each player?

In [20]:
query = """
WITH first_last AS (
SELECT playerID, min(yearID) as First_Year, max(yearID) as Last_Year
FROM fielding
GROUP BY playerID)

SELECT people.nameFirst, people.nameLast, first_last.First_Year, first_last.Last_Year
FROM first_last LEFT JOIN people
ON first_last.playerID = people.playerID
"""

first_last_year = pd.read_sql_query(query, conn)
first_last_year

,nameFirst,nameLast,First_Year,Last_Year
0,David,Aardsma,2004,2015
1,Hank,Aaron,1954,1976
2,Tommie,Aaron,1962,1971
3,Don,Aase,1977,1990
4,Andy,Abad,2001,2003
...,...,...,...,...
19486,Frank,Zupo,1957,1961
19487,Paul,Zuvella,1982,1991
19488,George,Zuverink,1951,1959
19489,Dutch,Zwilling,1910,1916


<br>

#### 3. Who has played the most all star games?

In [32]:
query = """

WITH allstar AS (
SELECT playerID, COUNT(*) as Game_Count
FROM allstarfull
GROUP BY playerID)

SELECT people.nameFirst, people.nameLast, allstar.Game_Count
FROM allstar LEFT JOIN people
ON allstar.playerID = people.playerID
INNER JOIN 
    (SELECT playerID, MAX(Game_Count) OVER () AS Max_Games
    FROM allstar) maxgames
    ON allstar.playerID = maxgames.playerID and allstar.Game_Count = maxgames.Max_Games
"""

all_star_games = pd.read_sql_query(query, conn)
all_star_games

,nameFirst,nameLast,Game_Count
0,Hank,Aaron,24
1,Willie,Mays,24
2,Stan,Musial,24


##### Answer: There is a three way tie between who played the most all start games - Hank Aaron, Willie Mays, and Stan Musial

<br>

#### 4. Which school has generated the most distinct players? 

In [42]:
query = """

SELECT name_full AS School, COUNT(DISTINCT playerID) as Game_Count
FROM collegeplaying LEFT JOIN schools
ON collegeplaying.schoolID = schools.schoolID
GROUP BY name_full 
ORDER BY COUNT(DISTINCT playerID) DESC
LIMIT 5
"""

school = pd.read_sql_query(query, conn)
school

,School,Game_Count
0,University of Texas at Austin,107
1,University of Southern California,105
2,Arizona State University,101
3,Stanford University,86
4,University of Michigan,76


##### Answer: The University of Texas at Austin has generated the most distinct players

<br>

#### 5. Which players have the longest career? Assume that the debut and finalGame columns comprise the start and end, respectively, of a player's career. 

In [105]:
query = """


SELECT nameFirst, nameLast, CAST(debut AS REAL) AS debut, CAST(finalGame AS DATETIME) AS final
FROM people


"""

school = pd.read_sql_query(query, conn)
school.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19878 entries, 0 to 19877
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   nameFirst  19841 non-null  object 
 1   nameLast   19878 non-null  object 
 2   debut      19682 non-null  object 
 3   final      19682 non-null  float64
dtypes: float64(1), object(3)
memory usage: 621.3+ KB


<br>

#### 6. What is the distribution of debut months? 

In [96]:
query = """


SELECT nameFirst, nameLast, EXTRACT(MONTH FROM DATE debut) as debut, 
DATE(finalGame) as finalGame
FROM people


"""

school = pd.read_sql_query(query, conn)
school.info()

DatabaseError: Execution failed on sql '


SELECT nameFirst, nameLast, EXTRACT(MONTH FROM DATE debut) as debut, DATE(finalGame) as finalGame
FROM people


': near "FROM": syntax error

<br>

#### 7. What is the effect of table join order on mean salary for the players listed in the main (master) table? 

In [88]:
query = """
SELECT people.playerID, avg(salaries.salary) as AVG_Salaries 
FROM salaries LEFT JOIN people
ON salaries.playerID = people.playerID
GROUP BY people.playerID
"""

salary_first = pd.read_sql_query(query, conn)
salary_first

,playerID,AVG_Salaries
0,aardsda01,1.322821e+06
1,aasedo01,5.750000e+05
2,abadan01,3.270000e+05
3,abadfe01,7.532800e+05
4,abbotje01,2.462500e+05
...,...,...
5144,zumayjo01,7.011667e+05
5145,zuninmi01,5.138000e+05
5146,zupcibo01,1.436667e+05
5147,zuvelpa01,1.450000e+05


In [89]:
query = """
SELECT people.playerID, avg(salaries.salary) as AVG_Salaries 
FROM people LEFT JOIN salaries
ON people.playerID = salaries.playerID
GROUP BY people.playerID
"""

people_first = pd.read_sql_query(query, conn)
people_first

,playerID,AVG_Salaries
0,aardsda01,1.322821e+06
1,aaronha01,NaN
2,aaronto01,NaN
3,aasedo01,5.750000e+05
4,abadan01,3.270000e+05
...,...,...
19873,zupofr01,NaN
19874,zuvelpa01,1.450000e+05
19875,zuverge01,NaN
19876,zwilldu01,NaN


##### Answer: There are some players with no salary information. When the left join is done on salaries, those players disappear out of the resulting data set since they aren't in the salaries table and that left join will only keep rows in the salaries table. When the left join is done on the people table, the players stay in the resulting data set and just have null values since they are in the people table. 